# Advanced NLP Assignment 1

Name : G Rohit

Student ID : 19233292

# Task 1

Here I have read the data from the file line by line, each text input is then processed. 
processTweet function is called for each tweet input. This function cleans the text input provided. 

Here I have also printed the positive samples size and negative samples size. 
I have also printed vocabulary size of processed data and unprocessed data

In [1]:
import re, string
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) # list of stopwords to filter from the text
from nltk.tokenize import word_tokenize 

def processTweet(tweet):
    tweet = tweet.lower() # convert tweet to lower case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) # remove the http and other links from the tweet
    tweet = tweet.replace(":p","") # remove the :p smiley from the tweet
    tweet = tweet.replace("--"," ") # remove the sequence '--' from the tweet
    tweet= tweet.translate(str.maketrans('', '', string.punctuation)) # remove all puntuations from the tweet
    tweet = re.sub('@[^\s]+','',tweet) # remove all the @ mentions from the tweet
    tweet = re.sub('[\s]+', ' ', tweet) # replace extra spaces from tweet to a single space
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # replace hashtag word with only just word
    # remove stopwords from the tweet
    word_tokens = word_tokenize(tweet) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    tweet = " ".join(filtered_sentence)
    tweet = tweet.strip('\'"') # remove quotes from the starting and ending of the tweet
    return tweet

file = open("SemEval2018-T3-train-taskA.txt",'rt',encoding='utf-8') # reading dataset
header = True
data, data_labels  = [], []
vocab, unprocessed_vocab = [], []
pos_examples = []
neg_examples  = []
for line in file : 
    if header : 
        header = False
        continue
    line = line.strip().split("\t")
    tweet = processTweet(line[2])
    unprocessed_vocab.extend(line[2].split())
    vocab.extend(tweet.split())
    data.append(tweet)
    data_labels.append(int(line[1]))
vocab_size = len(set(vocab))
print("Unprocessed Vocab size: ", len(set(unprocessed_vocab)))
print("Vocab size: ", vocab_size)
print("Positive Examples: ", data_labels.count(1))
print("Negative Examples: ",data_labels.count(0))

Unprocessed Vocab size:  18576
Vocab size:  12199
Positive Examples:  1911
Negative Examples:  1923


# Task 2

Here I have used scikit learn's train_test_split to split the dataset. I have used 80% of the data as training data and 20% data as the testing data. I have the used the ratio of 8:2 for splitting the dataset as this will provide us enough data to train the model.

Implemented function performance_evaluation. This function prints out the classification report as well as accuracy for a set of predictions. This functions calls scikit learn's functions to print out the classification report and accuracy

I have used CountVectorizer function to convert the text to feature vectors. This feature vector is splitted to tran and test data so that it can be used later while creating a log linear model.

In [2]:
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

#using CountVectorizer method to get feature vector od the dataset
countVectorizer = CountVectorizer(analyzer = 'word')
vocab_features = countVectorizer.fit_transform(data)
feature_vector = vocab_features.toarray()
X_train, X_test, y_train, y_test  = train_test_split(feature_vector, data_labels, train_size=0.80, random_state=1)

#function to calculate accuracy, precision, recall, f1-score
def performance_evaluation(y_true,y_pred):
    print('Accuracy:\t{:0.1f}%'.format(accuracy_score(y_true,y_pred)*100))   
    #classification report
    print('\n')
    print(classification_report(y_true, y_pred)) # prints precision, recall and f1 score
    #confusion matrix
    confmat = confusion_matrix(y_true, y_pred)
    print("The Confusion matrix is: -")
    print(confmat)

# Task 3

Here I have used scikit learn's LogisticRegression model to classify the tweet. This model uses a simple log linear activation function to fit on the dataset.

The input to this model is the CountVectorizer feature vector. Predcitions from this model is then sent to performance_evaluation function which was defined earlier. This will print out the classification summary of this model.

Accuracy of this model : 60.6%

In [3]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='lbfgs')
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)
performance_evaluation(y_test, y_pred)

Accuracy:	60.6%


              precision    recall  f1-score   support

           0       0.58      0.68      0.62       369
           1       0.64      0.54      0.59       398

    accuracy                           0.61       767
   macro avg       0.61      0.61      0.61       767
weighted avg       0.61      0.61      0.60       767

The Confusion matrix is: -
[[250 119]
 [183 215]]


# Task 4 

Here I have written three functions which will be passed as custom metrics to the Keras model for evalation. These function calculate Precision, Recall, F1-Score of the keras models.

In [41]:
from keras import backend as K

def calc_recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def calc_precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def calc_f1(y_true, y_pred):
    precision = calc_precision(y_true, y_pred)
    recall = calc_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Here I am building a Keras model for a simple RNN LSTM model. 
The input to this model is keras tokenizer. This converts each word in the vocabulary to a unique id. Then the dataset is replaced with the unique ids. After tokenizing the dataset, padding is done with(max length as 20). This will make sure that each tweet is of the same length. Padding is done to starting of the input. This padded input sequence is then again splitted into 8:2 ratio i.e. divided into test and train dataset. Model is trained on the train set. After that the model is tested against the test dataset and accuracy, precision, recall, f1-score is then printed. 
I have used embedding size as 128. I have also used dropout in the LSTM layer

Accuracy of this model : 61.3

In [46]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
import numpy as np
#creating keras tokenizer 
keras_tokenizer = Tokenizer()
keras_tokenizer.fit_on_texts(data)
X = keras_tokenizer.texts_to_sequences(data)
#converting data to sequences of same length
X = sequence.pad_sequences(X, maxlen=20)
y = np.asarray(data_labels)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 1234)
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size+1, 128))
rnn_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', calc_recall, calc_precision, calc_f1])
print('Training on data...')
rnn_model.fit(X_train, y_train,batch_size=32,epochs=5,validation_data=(X_test, y_test))
score, acc, recall, precision, f1_score = rnn_model.evaluate(X_test, y_test,batch_size=32)
print()
print('Rnn Model accuracy:', acc*100)
print('Rnn Model Recall:', recall)
print('Rnn Model Precison:', precision)
print('Rnn Model F1-score:', f1_score)

Training on data...


C:\Users\grohi\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3067 samples, validate on 767 samples
Epoch 1/5
3067/3067 [==============================] - 8s 3ms/step - loss: 0.6870 - accuracy: 0.5465 - calc_recall: 0.6227 - calc_precision: 0.5692 - calc_f1: 0.5365 - val_loss: 0.6653 - val_accuracy: 0.5671 - val_calc_recall: 0.7777 - val_calc_precision: 0.5368 - val_calc_f1: 0.6291_precisio
Epoch 2/5
3067/3067 [==============================] - 6s 2ms/step - loss: 0.4602 - accuracy: 0.8053 - calc_recall: 0.8024 - calc_precision: 0.8138 - calc_f1: 0.7993 - val_loss: 0.7094 - val_accuracy: 0.6102 - val_calc_recall: 0.6582 - val_calc_precision: 0.5894 - val_calc_f1: 0.6133
Epoch 3/5
3067/3067 [==============================] - 6s 2ms/step - loss: 0.1476 - accuracy: 0.9501 - calc_recall: 0.9499 - calc_precision: 0.9510 - calc_f1: 0.9491 - val_loss: 1.1101 - val_accuracy: 0.6206 - val_calc_recall: 0.5791 - val_calc_precision: 0.6159 - val_calc_f1: 0.5873
Epoch 4/5
3067/3067 [==============================] - 6s 2ms/step - loss: 0.0525 - accur

# Task 5 

As a part of improvement I am implementing a Convolution 1D model for text classification. The concept of convolution is used for images mainly where spatial features matter. For example nose is always near to an eye in an image of a person. So nose and eye is spatially related to each other. These features are detected very nicely with convolution layer. 

Simlarly when dealing with Text classification, words which are near to each other are also supposed to relate each other. Concept of n-gram says the words occuring together have some meaning to it. If two word occur together that means they have some relationship between them, this should also be captured as features. Convolution 1D will help us to acheive this as it learns automatically about spatial features between words. 

With this idea I have a built a model in Kearas. This model uses Conv1D layer. 

Accuracy of this model : 63.5%

The accuracy, precision, recall and F1-Score has increased from simple RNN LSTM model. This shows that this model will be helpful for text classfication tasks.

In [47]:
from keras.layers import Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
#hyper parameters: 
batch_size = 32
embedding_dims = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

#building the model
#input layer
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size+1, embedding_dims, input_length=20))
cnn_model.add(Dropout(0.2))

# create a Conv1D layer and then a plain hidden dense with non-linear activation
cnn_model.add(Conv1D(filters,kernel_size, padding='valid', activation='relu', strides=1))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(hidden_dims))
cnn_model.add(Dropout(0.2))
cnn_model.add(Activation('relu'))

#output layer with sigmoid activation
cnn_model.add(Dense(1))
cnn_model.add(Activation('sigmoid'))

cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', calc_recall, calc_precision, calc_f1])
#training the model
cnn_model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))
#testing
score, acc, recall, precision, f1_score = cnn_model.evaluate(X_test, y_test,batch_size=batch_size)
print('Conv 1D Model accuracy:', acc*100)
print('Conv 1D Model Recall:', recall)
print('Conv 1D Model Precison:', precision)
print('Conv 1D Model F1-score:', f1_score)

C:\Users\grohi\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3067 samples, validate on 767 samples
Epoch 1/5
3067/3067 [==============================] - 5s 2ms/step - loss: 0.6909 - accuracy: 0.5276 - calc_recall: 0.5696 - calc_precision: 0.5571 - calc_f1: 0.4872 - val_loss: 0.6750 - val_accuracy: 0.5854 - val_calc_recall: 0.1945 - val_calc_precision: 0.7933 - val_calc_f1: 0.3052
Epoch 2/5
3067/3067 [==============================] - 4s 1ms/step - loss: 0.4945 - accuracy: 0.7864 - calc_recall: 0.7727 - calc_precision: 0.8004 - calc_f1: 0.7727 - val_loss: 0.6921 - val_accuracy: 0.6245 - val_calc_recall: 0.6764 - val_calc_precision: 0.6001 - val_calc_f1: 0.6271
Epoch 3/5
3067/3067 [==============================] - 4s 1ms/step - loss: 0.1194 - accuracy: 0.9619 - calc_recall: 0.9630 - calc_precision: 0.9618 - calc_f1: 0.9610 - val_loss: 0.9708 - val_accuracy: 0.6284 - val_calc_recall: 0.6315 - val_calc_precision: 0.6104 - val_calc_f1: 0.6147
Epoch 4/5
3067/3067 [==============================] - 4s 1ms/step - loss: 0.0242 - accuracy: 0.99